In [1]:
import torch
from torchvision import datasets, transforms
import numpy as np
from numpy import genfromtxt
from torch.utils.data import Dataset
import cv2
import time
from preprocessing import preprocess_rgb, preprocess_flow
from torch.utils.data import DataLoader

In [20]:
class VideoDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,
                 csv_file, 
                 root_dir,
                 stream='rgb',
                 transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv_data = genfromtxt(csv_file, delimiter=',', dtype=str)
        self.root_dir = root_dir
        self.stream = stream
        self.transform = transform

    def __len__(self):
        return self.csv_data.shape[0]
        
        
    def __getitem__(self, idx):
        video_id = self.csv_data[idx][0]
        label = self.csv_data[idx][1]
        sample = {'videos': {'video_rgb': torch.zeros(224, 224), 'video_flow': torch.randn(224, 224)}, 'label': label}
        
        if self.stream == 'rgb' or self.stream == 'joint':
            sample['videos']['video_rgb'] = torch.from_numpy(preprocess_rgb(self.root_dir + video_id + '.mp4'))
        if self.stream == 'flow' or self.stream == 'joint':
            sample['videos']['video_flow'] = preprocess_flow(self.root_dir + video_id + '.mp4')
            
        return sample

In [3]:
loader = DataLoader(dataset=VideoDataset("./data/video_dataset.csv", "./data/", stream='rgb'),
                    batch_size=1,
                    shuffle=False,
                    num_workers=1)

In [4]:
for idx, batch in enumerate(loader):
    print ('---------------------------------')
    print (batch['videos']['video_rgb'].size())

---------------------------------
torch.Size([1, 1, 150, 224, 224, 3])
---------------------------------
torch.Size([1, 1, 150, 224, 224, 3])
---------------------------------
torch.Size([1, 1, 150, 224, 224, 3])
---------------------------------
torch.Size([1, 1, 150, 224, 224, 3])
---------------------------------
torch.Size([1, 1, 150, 224, 224, 3])
---------------------------------
torch.Size([1, 1, 134, 224, 224, 3])


In [18]:
vid2 = batch['videos']['video_rgb'].view(134,224,224,3).numpy()

In [19]:
vid2.shape

(134, 224, 224, 3)

In [ ]:
plt.imshow()

In [8]:
from PIL import Image
from array2gif import write_gif

# This script runs a gym environment and creates a video from the numpy array.
# Also a good example of how to create a video from a numpy array, using OpenCV

frame = vid2[0,:,:,:]


N_steps = 134
#frames = np.zeros(frame.shape[0], frame.shape[1], frame.shape[2], N_steps)
#frames[:,:,:,0] = frame
frames = vid2

# frame dimensions
# 400 x 600 x 3

#frames = np.ndarray(shape=(frame.shape[0], frame.shape[1], frame.shape[2], N_steps), dtype=int16)
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('simple_output.avi',fourcc, 20.0, (frame.shape[1], frame.shape[0]))
#out = cv2.VideoWriter('simple_output.avi', -1, 20.0, (frame.shape[1], frame.shape[0]))

#writer = cvCreateVideoWriter('simple_output.avi', -1, 20.0, (frame.shape[0], frame.shape[1]), is_color=1)

i = 1
for i in range(N_steps - 1):
    frame = vid2[i,:,:,:]

    frame2 = np.reshape( frame, (frame.shape[2], frame.shape[0], frame.shape[1]) ) 
    frames[:,:,:, i] = frame2
    frames.append(frame2)
    frames.append(frame[:,:,0])
    out.write(frame)
    #cvWriteFrame(out, frame)
    i+=1

print ("Environment closed")
out.release()
#im = Image.fromarray(frame)
#im.save("frame.jpeg")
write_gif(frames, 'rgb.gif', fps=24)


ValueError: could not broadcast input array from shape (3,224,224) into shape (134,224,224)

In [10]:
import matplotlib.pyplot as plt

In [17]:
plt.show(vid2[100,:,:,1].all())